In [1]:
#################################
# FIRST EXPERIMENT
# COMPARING NAVIGATION STRATEGIES WITHOUT PROVENANCE

In [2]:
# Imports
import sqlite3
import time
from graphviz import Graph

from pyalgdb.validity import Validity 
from pyalgdb.node import Node
from pyalgdb.provenance_enhancement import ProvenanceEnhancement
from pyalgdb.execution_tree_creator import ExecTreeCreator
from pyalgdb.top_down import TopDown
from pyalgdb.heaviest_first import HeaviestFirst
from pyalgdb.visualization import Visualization
from pyalgdb.single_stepping import SingleStepping
from pyalgdb.divide_and_query import DivideAndQuery

In [3]:
EXAMPLE_NAME = 'age-avg'

In [4]:
NOW2_SQLITE_PATH = 'C:/Users/linha/Desktop/ws/py-scripts-examples/' + EXAMPLE_NAME + '/.noworkflow/db.sqlite'

In [5]:
ANSWER_FILE_PATH = 'aux_files/' + EXAMPLE_NAME + '-answers.json'

In [6]:
GENERATE_TREES = True

In [7]:
CURSOR = sqlite3.connect(NOW2_SQLITE_PATH).cursor()

In [8]:
creator = ExecTreeCreator(CURSOR)
navs = [SingleStepping, TopDown, HeaviestFirst, DivideAndQuery] 

In [9]:
for nav in navs:
    exec_tree = None
    exec_tree = creator.create_exec_tree()
    nav_instance = nav(exec_tree, True, ANSWER_FILE_PATH)
    nav_instance.navigate()
    print(nav_instance.__class__.__name__+" experiment finished: "+str(nav_instance.sequence_num)+" steps.")
    if GENERATE_TREES:
        vis = Visualization(exec_tree)
        vis.view_exec_tree(str(id(exec_tree)))
    

SingleStepping experiment finished: 30 steps.
TopDown experiment finished: 9 steps.
HeaviestFirst experiment finished: 8 steps.
DivideAndQuery experiment finished: 7 steps.


In [10]:
#################################
# SECOND EXPERIMENT
# COMPARING NAVIGATION STRATEGIES WITH PROVENANCE PRUNE

In [11]:
creator = ExecTreeCreator(CURSOR)
navs = [SingleStepping, TopDown, HeaviestFirst, DivideAndQuery] 

In [12]:
class CustomVisualization(Visualization):

    PROVENANCE_COLOR = 'dodgerblue'
    PROVENANCE_NODE_COLOR = 'lightblue'
    PROV_PRUNED_NODE = 'grey55'
    
    def navigate(self, node:Node):
        chds = node.childrens
        for n in chds:
            self.graph.edge(str(node.ev_id), str(n.ev_id), None, dir='forward')
            if n.validity == Validity.INVALID:
                self.graph.node(str(n.ev_id), str(n.get_name()), fillcolor=self.INVALID_COLOR, style='filled')
            elif n.validity == Validity.VALID: 
                self.graph.node(str(n.ev_id), str(n.get_name()), fillcolor=self.VALID_COLOR, style='filled')
            elif n.validity == Validity.UNKNOWN:  
                self.graph.node(str(n.ev_id), str(n.get_name()))
            elif n.validity is Validity.NOT_IN_PROV:
                self.graph.node(str(n.ev_id), str(n.get_name()), fillcolor=self.PROV_PRUNED_NODE, style='filled')

        if len(chds) > 0:
            g = Graph()
            for c in chds:
                g.node(str(c.ev_id))
            g.graph_attr['rank']='same'
            self.graph.subgraph(g)

        for n in chds: 
            self.navigate(n)


In [13]:
for nav in navs:
    
    exec_tree = None
    exec_tree = creator.create_exec_tree()
    
    prov = ProvenanceEnhancement(exec_tree, CURSOR)

    prov.enhance_all()    
        
    nav_instance = nav(exec_tree, True, ANSWER_FILE_PATH)
    nav_instance.provenance_prune()
    nav_instance.navigate()
    print(nav_instance.__class__.__name__+" experiment finished: "+str(nav_instance.sequence_num)+" steps.")
    if GENERATE_TREES:
        vis = CustomVisualization(exec_tree)
        vis.view_exec_tree(str(id(exec_tree)))

SingleStepping experiment finished: 11 steps.
TopDown experiment finished: 9 steps.
HeaviestFirst experiment finished: 7 steps.
DivideAndQuery experiment finished: 6 steps.
